In [272]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
t = pd.read_csv('/home/alloxa/travelhack6.csv')

In [ ]:
t = t.rename(columns={'Состав группы (Заявка) (Заявка)': 'Состав группы'})

In [396]:
t.isna().sum().sort_values()

Пол                       0
Город отправления         0
Глубина продаж            0
ИД клиента                0
Наименование тура         0
Сумма в $                 0
Состав группы             0
Младенцы                  0
Туристы, дети             0
visa                      0
Туристов                  0
Туристы, взрослые         0
Тип заявки                0
Ночей                     0
Год                       0
Дата начала тура          0
Дата бронирования         0
Заявка №                  0
Возраст клиента           0
Расстояние                0
День рождения           478
Страна тура             940
Вид тура                940
Звездность            99347
Тип питания           99347
Город Отеля          110421
Регион отеля         110421
dtype: int64

In [397]:
t['ИД клиента'].nunique()

193276

In [399]:
t['Страна тура'].value_counts()

Турция         124842
Кипр            17868
Россия           9538
Испания          7582
ОАЭ              7472
                ...  
Дания               6
Азербайджан         5
Литва               4
Норвегия            4
Сербия              1
Name: Страна тура, Length: 52, dtype: int64

In [ ]:
#замена ошибочных возраста и дата рождений NaN-ами. 
t['День рождения'][t['Возраст клиента']>100] = np.NaN
t['Возраст клиента'][t['Возраст клиента']>100] = np.NaN

In [ ]:
#Замена Nan возраста и даты рождения медианой с учетом звездности, страны тура и состава группы. 
for i in t['Звездность'].unique():
    for j in t['Вид тура'].unique():
        for k in t['Состав группы'].unique():
            t['Возраст клиента'][t['Возраст клиента'].isna() & (t['Звездность']==i) & (t['Вид тура']==j) & (t['Состав группы']==k)] = t['Возраст клиента'][(t['Звездность']==i) & (t['Вид тура']==j) & (t['Состав группы']==k)].median()


In [ ]:
#Замена оставшихся нанов медианой по всей выборке
t['Возраст клиента'] = t['Возраст клиента'].fillna(t['Возраст клиента'].median())

In [ ]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
geolocator = Nominatim(user_agent="Alloxa")

In [291]:
t['Расстояние'] = 0

In [269]:
t['Страна тура'].unique()

array(['Турция', 'Кипр', 'Россия', 'Абхазия', 'ОАЭ', 'Мальдивы',
       'Черногория', 'Греция', 'Круизы', 'Беларусь', 'Тунис', 'Испания',
       'Болгария', 'Италия', nan, 'Таиланд', 'Албания',
       'Доминиканская Республика', 'Иордания', 'Хорватия', 'Израиль',
       'Финляндия', 'Андорра', 'Чехия', 'Куба', 'Франция', 'Шри-Ланка',
       'Швеция', 'Вьетнам', 'Китай', 'Грузия', 'Индонезия', 'Португалия',
       'Венгрия', 'Латвия', 'Австрия', 'Япония', 'Великобритания',
       'Маврикий', 'Мексика', 'Южная Корея', 'Ирландия', 'Германия',
       'Эстония', 'Швейцария', 'Словения', 'Польша', 'Азербайджан',
       'Армения', 'Норвегия', 'Дания', 'Сербия', 'Литва'], dtype=object)

In [ ]:
#Заполняем Расстояние между городом отправления и страной назначения
for town_out in tqdm(t['Город отправления'].unique()):
    for cantry_in in t['Страна тура'].unique():
            cantry_in = 'Куба'
            try:
                location1 = geolocator.geocode(town_out)
                location2 = geolocator.geocode(cantry_in)
                coord_loc1 = (location1.latitude, location1.longitude)
                coord_loc2 = (location2.latitude, location2.longitude)
                t['Расстояние'][(t['Город отправления']==town_out)&(t['Страна тура']==cantry_in)] = round(geodesic(coord_loc1, coord_loc2).km)
            except:
                print('не удалось записать расстояние между '+town_out+' и '+cantry_in)
        

In [ ]:
#Заполняем Расстояние между городом отправления и  городом отеля (при отсутствии города отеля замена страной назначения)
for i in tqdm(t.index):
    if t.loc[i, 'Город Отеля'] != t.loc[i, 'Город Отеля']:
        if t.loc[i, 'Страна тура'] != t.loc[i, 'Страна тура']:
            t.loc[i, 'Расстояние'] = 0
        else:
            location1 = geolocator.geocode(t.iloc[i]['Город отправления'])
            location2 = geolocator.geocode(t.iloc[i]['Страна тура'])
            coord_loc1 = (location1.latitude, location1.longitude)
            coord_loc2 = (location2.latitude, location2.longitude)
            t.loc[i, 'Расстояние'] = round(geodesic(coord_loc1, coord_loc2).km)
    else:
        try:
            location1 = geolocator.geocode(t.iloc[i]['Город отправления'])
            location2 = geolocator.geocode(t.iloc[i]['Город Отеля'])
            coord_loc1 = (location1.latitude, location1.longitude)
            coord_loc2 = (location2.latitude, location2.longitude)
            t.loc[i, 'Расстояние'] = round(geodesic(coord_loc1, coord_loc2).km)
        except:
            if t.loc[i, 'Страна тура'] == t.loc[i, 'Страна тура']:
                location1 = geolocator.geocode(t.iloc[i]['Город отправления'])
                location2 = geolocator.geocode(t.iloc[i]['Страна тура'])
                coord_loc1 = (location1.latitude, location1.longitude)
                coord_loc2 = (location2.latitude, location2.longitude)
                t.loc[i, 'Расстояние'] = round(geodesic(coord_loc1, coord_loc2).km)
            else:
                t.loc[i, 'Расстояние'] = 0
    
        

In [391]:
distance = t['Расстояние']

In [393]:
distance.to_csv("distance.csv", index = True)